# N05: Gap-Based Strategy Rules for F1

## 1. Introduction and Objective


Gap analysis is a critical component of Formula 1 race strategy. The time difference between cars (measured in seconds) directly influences strategic decisions like undercuts, overcuts, and defensive pit stops. This notebook implements rules that use these gaps to make strategic recommendations.


### Why Gap Analysis Matters

* **Undercut Opportunities:** When a car is close behind another (~1-2s), pitting earlier might allow overtaking through fresher tires
* **Overcut Potential:** Sometimes staying out longer works better, especially when the leading car has clean air
* **Defensive Strategy**: Teams must react when competitors attempt undercuts
* **Traffic Management**: Gaps determine whether a car will exit the pits into free air or traffic

### Our Objectives

Extract and process gap data from FastF1 (replacing our previous computer vision approach)
Analyze gap distributions to establish meaningful thresholds for strategic decisions
Implement four key rules:

1. **Undercut Opportunity Rule**: Identify when an undercut might succeed
2. **Defensive Pit Stop Rule**: Protect position against undercut attempts
3. **Overcut Strategy Rule:** Recognize when staying out longer is advantageous
4. **Traffic Management Rule**: Avoid exiting pit stops into traffic



These rules will complement our existing components (tire degradation, lap time analysis, and radio communication) to create a comprehensive F1 strategy system.

In [1]:
__author__ = "Víctor Vega Sobral"

---